In [2]:
import os

In [3]:
%pwd

'h:\\git_kdc_mlflow\\kdc_mlflow\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'h:\\git_kdc_mlflow\\kdc_mlflow'

In [6]:
os.environ['MLFLOW_TRACKING_URI']="https://dagshub.com/sandmanGPU/kdc_mlflow.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME']="sandmanGPU"
os.environ['MLFLOW_TRACKING_PASSWORD']="170fae55eed33119e40cf121df270c286e09c95f"


In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_to_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [10]:
from src.cnnClassifier.constants import*
from src.cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self)->EvaluationConfig:
        eval_config = EvaluationConfig(
            path_to_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/sandmanGPU/kdc_mlflow.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
        

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from src.cnnClassifier.utils.common import save_json

In [15]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_to_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store=urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy":self.score[1]}
            )

            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [16]:
try:
    config= ConfigurationManager()
    eval_config=config.get_evaluation_config()
    evaluation=Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
    raise e
    

[2023-10-09 06:31:52,052: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-10-09 06:31:52,054: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-09 06:31:52,056: INFO: common: created directory at: artifacts]
Found 139 images belonging to 2 classes.
9/9 [==============================] - 22s 2s/step - loss: 10.1576 - accuracy: 0.5180
[2023-10-09 06:32:15,034: INFO: common: json file saved at: scores.json]


AttributeError: module 'mlflow' has no attribute 'set_registry'